In [1]:
import torch
import cv2

print("Success! Libraries are loaded from the anpr_env.")
print(f"PyTorch version: {torch.__version__}")

Success! Libraries are loaded from the anpr_env.
PyTorch version: 2.8.0+cu126


In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # nano model for 4GB VRAM

results = model.train(
    data='../data/YOLO_dataset/data.yaml',
    epochs=200,
    imgsz=800,      # safe resolution
    batch=2,        # reduce to avoid OOM
    device=0,
    workers=2,
    optimizer='AdamW',
    amp=True         # mixed precision
)


Ultralytics 8.3.193  Python-3.11.13 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/YOLO_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=1


0: 512x640 1 number_plate, 59.9ms
Speed: 4.8ms preprocess, 59.9ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 640)
Detected Plate Text: Mho1av8866, Confidence: 0.78


<Figure size 640x480 with 1 Axes>

In [12]:
import cv2
from ultralytics import YOLO
import easyocr
import re 

# 1. Load the model you already trained on images
plate_detector = YOLO('../models/best.pt')

# Create a character whitelist for EasyOCR
VALID_CHARS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
reader = easyocr.Reader(['en'])

# Compile a regex pattern for Indian license plates
plate_format_pattern = re.compile(r'^[A-Z]{2}[0-9]{1,2}[A-Z]{0,2}[0-9]{1,4}$')

# Load allowed plates from a file into a set for fast lookup
allowed_plates_path = 'allowed_plates.txt'
allowed_plates = set()
try:
    with open(allowed_plates_path, 'r') as f:
        allowed_plates = {line.strip().upper() for line in f}
    print("Successfully loaded allowed plates.")
except FileNotFoundError:
    print(f"Warning: '{allowed_plates_path}' not found. No plates will be marked as ALLOWED.")

# 3. Path to your video file
video_path = '../data/ANPRshi.mp4'

# Create a set to store unique plate numbers
detected_plates_set = set()

# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
else:
    # Loop through the video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            detections = plate_detector(frame)[0]

            for detection in detections.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = detection
                
                plate_img = frame[int(y1):int(y2), int(x1):int(x2)]
                
                ocr_result = reader.readtext(plate_img, allowlist=VALID_CHARS)
                
                if ocr_result:
                    plate_text = ocr_result[0][1].upper().replace(' ', '')
                    
                    if plate_format_pattern.match(plate_text):
                        detected_plates_set.add(plate_text)
                        
                        # --- This part still shows both allowed/not-allowed on screen ---
                        # --- To change this, see the optional note below the code ---
                        display_text = plate_text
                        color = (0, 0, 255) # Red for not allowed

                        if plate_text in allowed_plates:
                            display_text = f"{plate_text} ALLOWED"
                            color = (0, 255, 0) # Green for allowed
                        
                        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                        cv2.putText(frame, display_text, (int(x1), int(y1) - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Resize the frame for display
            scale_percent = 40 
            width = int(frame.shape[1] * scale_percent / 100)
            height = int(frame.shape[0] * scale_percent / 100)
            dim = (width, height)
            
            resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
            
            cv2.imshow('ANPR Video Processing', resized_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

# Release everything
cap.release()
cv2.destroyAllWindows()
print("Video processing finished.")

# --- CHANGE HERE: Only print the plates that are in the allowed list ---
print("\n--- Summary of ALLOWED Plates Detected ---")
# Convert the set to a sorted list for clean output
sorted_plates = sorted(list(detected_plates_set))

# Counter to check if any allowed plates were found
found_allowed_plates = False

for plate in sorted_plates:
    # Only print the plate if it's in the allowed_plates set
    if plate in allowed_plates:
        print(plate)
        found_allowed_plates = True

if not found_allowed_plates:
    print("No allowed license plates were detected.")
print("------------------------------------------")

Successfully loaded allowed plates.

0: 384x640 (no detections), 10.0ms
Speed: 2.6ms preprocess, 10.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 8.6ms
Speed: 1.8ms preprocess, 8.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 13.2ms
Speed: 1.7ms preprocess, 13.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 8.6ms
Speed: 1.8ms preprocess, 8.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 9.3ms
Speed: 2.3ms preprocess, 9.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 9.3ms
Speed: 1.7ms preprocess, 9.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 13.5ms
Speed: 1.6ms preprocess, 13.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 number_plate, 10.4ms
Speed: 2.0ms